<a href="https://colab.research.google.com/github/Marfall/MusicAnalyzer/blob/main/MusicAnalyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Проектная работа "Классификация музыкальных жанров"

## Проверка среды и подключение Google Диска

In [8]:
# Определяем, работаем ли мы в Google Colab
import sys
IN_COLAB = 'google.colab' in sys.modules

print("✅ Среда выполнения:", "Google Colab" if IN_COLAB else "Локальная")

if IN_COLAB:
    # Подключаем Google Диск (по желанию)
    from google.colab import drive
    drive.mount('/content/drive')
    print("✅ Google Диск подключён. Файлы доступны в /content/drive/MyDrive/")

    # Устанавливаем нужные библиотеки (в Colab они уже есть, но для уверенности)
    !pip install -q catboost scikit-learn pandas numpy matplotlib seaborn
    print("✅ Библиотеки установлены/обновлены.")
else:
    print("⚠️  Работаем локально. Убедитесь, что все библиотеки установлены.")

✅ Среда выполнения: Google Colab
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Google Диск подключён. Файлы доступны в /content/drive/MyDrive/
✅ Библиотеки установлены/обновлены.


## Импорт библиотек для работы с данными и визуализацией

In [9]:
# Импорт библиотек для работы с данными и визуализации
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Настройка стиля графиков
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)

# Импорт модулей sklearn
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix, roc_curve, auc
from sklearn.preprocessing import label_binarize

# Импорт CatBoost
from catboost import CatBoostClassifier

# Для распределений в RandomizedSearch
from scipy.stats import randint, uniform

# Игнорируем предупреждения
import warnings
warnings.filterwarnings('ignore')

# Функция для сохранения графиков
def save_fig(fig, filename):
    fig.savefig(filename, dpi=300, bbox_inches='tight')
    print(f"✅ График сохранён как {filename}")

## Загрузка датасета и первичный обзор

In [10]:
# Путь на Google Диске
import os
file_path = '/content/drive/MyDrive/Collab Documents/features_30_sec.csv'
if not os.path.exists(file_path):
  print(f"Файл не найден по пути {file_path}. Убедитесь, что вы подключили диск и файл лежит в MyDrive.")
else:
  print(f"✅ Файл найден на Google Диске: {file_path}")

# Загружаем данные
data = pd.read_csv(file_path)
print("\nРазмерность датасета")
print(data.shape)
# Выводим первые 5 строк, чтобы увидеть структуру
print("\nПервые 5 строк:")
print(data.head())

# Общая информация о данных: типы столбцов, количество непустых значений
print("\nИнформация о датасете:\n")
print(data.info())

✅ Файл найден на Google Диске: /content/drive/MyDrive/Collab Documents/features_30_sec.csv

Размерность датасета
(1000, 60)

Первые 5 строк:
          filename  length  chroma_stft_mean  chroma_stft_var  rms_mean  \
0  blues.00000.wav  661794          0.350088         0.088757  0.130228   
1  blues.00001.wav  661794          0.340914         0.094980  0.095948   
2  blues.00002.wav  661794          0.363637         0.085275  0.175570   
3  blues.00003.wav  661794          0.404785         0.093999  0.141093   
4  blues.00004.wav  661794          0.308526         0.087841  0.091529   

    rms_var  spectral_centroid_mean  spectral_centroid_var  \
0  0.002827             1784.165850          129774.064525   
1  0.002373             1530.176679          375850.073649   
2  0.002746             1552.811865          156467.643368   
3  0.006346             1070.106615          184355.942417   
4  0.002303             1835.004266          343399.939274   

   spectral_bandwidth_mean  spectra

### Краткая информация по датасету:

- Датасет содержит признаки признаки, описывающие ритм, тембр, частотный состав и динамику музыки – именно по ним модель учится различать жанры.



## Обзор данных (EDA):

### Проверка пропусков и дубликатов

In [14]:
# Проверяем наличие пропусков в каждом столбце
print("Пропуски в столбцах:\n")
print(data.isnull().sum())

# Проверяем полностью дублированные строки
duplicates = data.duplicated().sum()
print(f"\nКоличество дублированных строк: {duplicates}\n")

# Проверяем дубликаты в названиях файлов (должны быть уникальны)
duplicate_filenames = data['filename'].duplicated().sum()
print(f"Количество дубликатов в названиях файлов: {duplicate_filenames}")

Пропуски в столбцах:

filename                   0
length                     0
chroma_stft_mean           0
chroma_stft_var            0
rms_mean                   0
rms_var                    0
spectral_centroid_mean     0
spectral_centroid_var      0
spectral_bandwidth_mean    0
spectral_bandwidth_var     0
rolloff_mean               0
rolloff_var                0
zero_crossing_rate_mean    0
zero_crossing_rate_var     0
harmony_mean               0
harmony_var                0
perceptr_mean              0
perceptr_var               0
tempo                      0
mfcc1_mean                 0
mfcc1_var                  0
mfcc2_mean                 0
mfcc2_var                  0
mfcc3_mean                 0
mfcc3_var                  0
mfcc4_mean                 0
mfcc4_var                  0
mfcc5_mean                 0
mfcc5_var                  0
mfcc6_mean                 0
mfcc6_var                  0
mfcc7_mean                 0
mfcc7_var                  0
mfcc8_mean           

### Промежуточный вывод по качеству данных:

Проведена первичная проверка целостности датасета. Результаты свидетельствуют о высоком качестве исходных данных:

- ✅ **Пропуски отсутствуют** — все признаки имеют корректные значения, нет NaN или пустых записей.
- ✅ **Дубликаты строк не обнаружены** — метод `data.duplicated().sum()` вернул 0, что подтверждает уникальность каждой записи.
- ✅ **Названия треков уникальны** — каждый файл имеет своё оригинальное имя, повторов нет.

Таким образом, датасет можно считать **полностью очищенным от структурных ошибок** и готовым к дальнейшему анализу и моделированию. 📈